### This script will set up, simulate and gather all results for all cell rotations, for different model and/or field instantiations. The available settings are: 

**A:** Passive models with axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />
**B:** Passive models without axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />

**C:** Active models without axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />
**D:** Active models without axons. Subthreshold simulation. Point field 50 um away from soma. <br />
**E:** Passive models without axons. Subthreshold simulation. Point field 50 um away from soma. <br />

**F:** Active models without axons. Suprathreshold simulation with calibrated DC injection. Plane field 2.5 mm away from soma. <br />
**G:** Active models without axons. Suprathreshold simulation with calibrated DC injection. Point field 50 um away from soma. <br />

**H:** Active models without axons. Suprathreshold simulation without ES. <br />
**I:** Active models with custom passive axons with Cm = A. Suprathreshold simulation without ES. <br />
**J:** Active models with custom passive axons with Cm = A/4. Suprathreshold simulation without ES. <br />
**K:** Active models with custom passive axons with Cm = A/8. Suprathreshold simulation without ES. <br />

**L:** Active models with custom active axons with dendrite parameters. Suprathreshold simulation without ES. <br />
**M:** Active models with custom active axons with soma parameters. Suprathreshold simulation without ES. <br />
**N:** Active models with custom active axons with dendrite parameters all HoF. Suprathreshold simulation without ES. <br />

**O:** Active sticks with dendrite parameters all HoF. Pulse ES stimulation during rest. <br />
**P:** Active complex stick with dendrite parameters all HoF. Pulse ES stimulation during rest. <br />

In [1]:
Selected_Sweep = 'P'

simulations_in_parallel = 26
model_processing = 'csmc_allactive'

if Selected_Sweep == 'A':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected With/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [1,3,5,10,30,50,100,300,500,1000] # uA
    Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
elif Selected_Sweep == 'B':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [1,3,5,10,30,50,100,300,500,1000] # uA
    Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
elif Selected_Sweep == 'C':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
elif Selected_Sweep == 'D':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,8,28,140]
elif Selected_Sweep == 'E':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,8,28,140]
elif Selected_Sweep == 'F':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = True
    Custom = False
    allHOF = False
    SuP_amplitudes = [0.0,10,100,500] # uA
    SuP_frequencies =[8,28,140] # Hz
elif Selected_Sweep == 'G':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = True
    Custom = False
    allHOF = False
    SuP_amplitudes_P = [0.0,0.05,0.1,0.2] # uA
    SuP_frequencies =[8,28,140] # Hz
elif Selected_Sweep == 'H':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = None
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'I':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'A'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'J':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'A/4'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'K':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'A/8'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'L':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'D'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'M':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'S'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'N':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = True
    axon_parameters = 'D'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'O':
    model_file = False
    swc_path = None
    short = True
    off = False
    supra = False
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = True
    axon_parameters = 'D'
    SuP_amplitudes = [-100,100] # uA
    SuP_frequencies =[0.1] # ms
    Manual_picked_model_list = None
elif Selected_Sweep == 'P':
    model_file = False
    swc_path = None
    short = False
    off = False
    supra = False
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = True
    axon_parameters = 'D'
    SuP_amplitudes = [-100,100] # uA
    SuP_frequencies =[0.1] # ms
    Manual_picked_model_list = None

##### Set up directories

In [2]:
import shutil, os, json
import pandas as pd

if swc_path is None:
    morphfiles = os.listdir('../Required_Files/Models/SWCs connected Without/')
else:
    morphfiles = os.listdir(swc_path)

inhpath = '../Required_files/Models/JSON Format/Inh Models/'
excpath = '../Required_files/Models/JSON Format/Exc Models/'
inhtypes = os.listdir(inhpath)
exctypes = os.listdir(excpath)

allmodels = []
for i in inhtypes:
    for j in os.listdir(inhpath + i):
        allmodels.append(inhpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

for i in exctypes:
    for j in os.listdir(excpath + i):
        allmodels.append(excpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

current_vals = pd.read_csv('../Required_files/Calibrated.csv')
current_vals = current_vals.sort_values('Target', ascending=True).reset_index(drop=True).reset_index()
current_vals = current_vals.set_index('cell_id')

simulation_no = 0
for i in morphfiles:
    if int(i.split('rmed_')[1].split('.sw')[0]) not in range(0,360,Rotation_Steps):
        continue
    if int(i.split('_')[2]) not in current_vals.index and Only_Calibrated:
        continue
    if Manual_picked_model_list is not None:
        if int(i.split('_')[2]) not in Manual_picked_model_list:
            continue
    if supra:
        frequencies = SuP_frequencies
    else:
        if Custom:
            frequencies = SuP_frequencies
        else:
            frequencies = Sub_frequencies
    for f in frequencies:
        if supra:
            if point:
                amplitudes = SuP_amplitudes_P
            else:
                amplitudes = SuP_amplitudes
        else:
            if Custom:
                amplitudes = SuP_amplitudes
            else:
                amplitudes = Sub_amplitudes
        for c in amplitudes:
            if allHOF:
                hof_range = 20
            else:
                hof_range = 1
            for hof in range(hof_range):
                full_path = os.getcwd().replace('\\','/') + '/Simulations/' + Selected_Sweep + '/SIM_' + \
                str(simulation_no)
                if os.path.exists(full_path):
                    if os.path.isfile(full_path + '/Data.csv'):
                        simulation_no += 1
                        continue
                    else:
                        shutil.rmtree(full_path)
                os.makedirs(full_path)
                if swc_path is None:
                    if short:
                        shutil.copyfile('../Required_Files/Models/Stick.swc', full_path + '/morph.swc')
                    else:
                        shutil.copyfile('../Required_Files/Models/StickFluctuating.swc', full_path + '/morph.swc')
                else:
                    shutil.copyfile(swc_path + i, full_path + '/morph.swc')
                shutil.copyfile('../Required_Files/Simulate.py', full_path + '/simulate.py')
                shutil.copyfile('../Required_Files/cell_functions.py', full_path + '/cell_functions.py')
                shutil.copyfile('../Required_Files/nrnmech.dll', full_path + '/nrnmech.dll')
                if model_file:
                    shutil.copyfile(model_file, full_path + '/model.json')
                else:
                    for mod in allmodels:
                        if i.split("transformed")[0] in mod:
                            inpa = mod.replace('hof_0','hof_'+str(hof))
                            oupa = full_path + '/model.json'
                            with open(inpa, 'r') as infile:
                                parameters = json.load(infile)
                            del parameters['axon_morph']
                            if Custom:
                                if swc_path is None:
                                    parameters['conditions'][0]['erev'].append(parameters['conditions'][0]['erev'][0])
                                    parameters['conditions'][0]['erev'][-1]['section'] = 'axon'
                                    if 'A' in axon_parameters:   
                                        with open('../Required_Files/Models/JSON Format/A.json', 'r') as infile:
                                            passive_param = json.load(infile)
                                        axonlist = []
                                        gendict = passive_param['genome']
                                        for di in gendict:
                                            if di['section']=='soma':
                                                if '/' in axon_parameters:
                                                    di['value'] = di['value']/int(axon_parameters.split('/')[1])
                                                elif '*' in axon_parameters:
                                                    di['value'] = di['value']*int(axon_parameters.split('*')[1])
                                                axonlist.append(di.copy())
                                        for gi in range(len(axonlist)):
                                            axonlist[gi]['section']='axon'
                                            parameters['genome'].append(axonlist[gi])
                                    elif 'S' in axon_parameters:
                                        axonlist = []
                                        gendict = parameters['genome']
                                        for di in gendict:
                                            if di['section']=='soma':
                                                axonlist.append(di.copy())
                                        for gi in range(len(axonlist)):
                                            axonlist[gi]['section']='axon'
                                            parameters['genome'].append(axonlist[gi])
                                    elif 'D' in axon_parameters:
                                        axonlist = []
                                        gendict = parameters['genome']
                                        for di in gendict:
                                            if di['section']=='dend':
                                                axonlist.append(di.copy())
                                        for gi in range(len(axonlist)):
                                            axonlist[gi]['section']='axon'
                                            parameters['genome'].append(axonlist[gi])
                                elif ('Without' not in swc_path):
                                    parameters['conditions'][0]['erev'].append(parameters['conditions'][0]['erev'][0])
                                    parameters['conditions'][0]['erev'][-1]['section'] = 'axon'
                                    if 'A' in axon_parameters:   
                                        with open('../Required_Files/Models/JSON Format/A.json', 'r') as infile:
                                            passive_param = json.load(infile)
                                        axonlist = []
                                        gendict = passive_param['genome']
                                        for di in gendict:
                                            if di['section']=='soma':
                                                if '/' in axon_parameters:
                                                    di['value'] = di['value']/int(axon_parameters.split('/')[1])
                                                elif '*' in axon_parameters:
                                                    di['value'] = di['value']*int(axon_parameters.split('*')[1])
                                                axonlist.append(di.copy())
                                        for gi in range(len(axonlist)):
                                            axonlist[gi]['section']='axon'
                                            parameters['genome'].append(axonlist[gi])
                                    elif 'S' in axon_parameters:
                                        axonlist = []
                                        gendict = parameters['genome']
                                        for di in gendict:
                                            if di['section']=='soma':
                                                axonlist.append(di.copy())
                                        for gi in range(len(axonlist)):
                                            axonlist[gi]['section']='axon'
                                            parameters['genome'].append(axonlist[gi])
                                    elif 'D' in axon_parameters:
                                        axonlist = []
                                        gendict = parameters['genome']
                                        for di in gendict:
                                            if di['section']=='dend':
                                                axonlist.append(di.copy())
                                        for gi in range(len(axonlist)):
                                            axonlist[gi]['section']='axon'
                                            parameters['genome'].append(axonlist[gi])
                                elif ('Without' not in swc_path):
                                    parameters['conditions'][0]['erev'].append(parameters['conditions'][0]['erev'][0])
                                    parameters['conditions'][0]['erev'][-1]['section'] = 'axon'
                                    axonlist = []
                                    gendict = parameters['genome']
                                    for di in gendict:
                                        if di['section']=='dend':
                                            axonlist.append(di.copy())
                                    for gi in range(len(axonlist)):
                                        axonlist[gi]['section']='axon'
                                        gendict.append(axonlist[gi])
                                    parameters['genome'] = gendict
                            json_object = json.dumps(parameters, indent=4)
                            with open(oupa, 'w') as outfile:
                                outfile.write(json_object)
                            break
                if Custom:
                    inj_curr = 0.3
                    inj_std = 0.0
                    exp_rate = 0
                elif supra:
                    inj_curr = current_vals.loc[int(i.split('_')[2]),'Mean']
                    inj_std = current_vals.loc[int(i.split('_')[2]),'StD']
                    exp_rate = current_vals.loc[int(i.split('_')[2]),'PM']/2
                else:
                    inj_curr = 0
                    inj_std = 0
                    exp_rate = 200
                fin = open(full_path + '/simulate.py', "rt")
                cnt = 10
                lines = []
                for line in fin:
                    if cnt==10:
                        lines.append('base_dir = "' + full_path + '"\n')
                    elif cnt==9:
                        lines.append('inj_curr = ' + str(inj_curr) + '\n')
                    elif cnt==8:
                        lines.append('inj_std = ' + str(inj_std) + '\n')
                    elif cnt==7:
                        lines.append('exp_rate = ' + str(exp_rate) + '\n')
                    elif cnt==6:
                        lines.append('es_freq = ' + str(f) + '\n')
                    elif cnt==5:
                        lines.append('es_ampl = ' + str(c) + '\n')
                    elif cnt==4:
                        lines.append('mod_proc = "' + model_processing + '"\n')
                    elif cnt==3:
                        if point:
                            lines.append('point = 1\n')
                        else:
                            lines.append('point = 0\n')
                    elif cnt==2:
                        lines.append('run = 1\n')
                    elif cnt==1:
                        if Custom:
                            if axon_parameters is not None:
                                if supra:
                                    lines.append('returns = 3\n')
                                else:
                                    if off:
                                        lines.append('returns = 6\n')
                                    else:
                                        lines.append('returns = 5\n')
                            else:
                                lines.append('returns = 4\n')
                        elif supra:
                            lines.append('returns = 1\n')
                        else:
                            lines.append('returns = 0\n')
                    else:
                        lines.append(line)
                    cnt -= 1
                fin.close()
                fout = open(full_path + '/simulate.py', "wt")
                for line in lines:
                    fout.write(line)
                fout.close()

                simulation_no += 1
print (simulation_no)

8520


##### Simulate

In [3]:
import subprocess as sbp
import os, shutil, time

total_simulations = simulation_no
print(total_simulations)
processes = []
process_names = []
active_count = 0
process_index = 0

start_time = time.time()
while (active_count > 0) or (process_index < total_simulations):
    active_processes = processes.copy()
    active_process_names = process_names.copy()
    processes = []
    process_names = []
    for i, sbprc in enumerate(active_processes):
        poll = sbprc.poll()
        if poll is None:
            processes.append(sbprc)
            process_names.append(active_process_names[i])
        else:
            active_count -= 1
            print ('Finished ' + active_process_names[i])
            if process_index >= total_simulations:
                print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
    if (active_count < simulations_in_parallel) and (process_index < total_simulations):
        process_name = 'SIM_'+str(process_index)
        if not os.path.isfile(os.getcwd().replace('\\','/') + '/SIMULATIONS/' + Selected_Sweep + \
                              '/' + process_name + "/Data.csv"):
            cmnd = ["python", 
                    os.getcwd().replace('\\','/') + '/SIMULATIONS/' + Selected_Sweep + '/' +  \
                    process_name + "/simulate.py"]
            p = sbp.Popen(cmnd)
            processes.append(p)
            process_names.append(process_name)
            active_count += 1
        process_index += 1
        print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
    if active_count >= simulations_in_parallel:
        time.sleep(2)
    
queue_time = time.time() - start_time
print("Total time " + str(queue_time/3600) + " hours")

8520
Active: 1, Pending: 8519
Active: 2, Pending: 8518
Active: 3, Pending: 8517
Active: 4, Pending: 8516
Active: 5, Pending: 8515
Active: 6, Pending: 8514
Active: 7, Pending: 8513
Active: 8, Pending: 8512
Active: 9, Pending: 8511
Active: 10, Pending: 8510
Active: 11, Pending: 8509
Active: 12, Pending: 8508
Active: 13, Pending: 8507
Active: 14, Pending: 8506
Active: 15, Pending: 8505
Active: 16, Pending: 8504
Active: 17, Pending: 8503
Active: 18, Pending: 8502
Active: 19, Pending: 8501
Active: 20, Pending: 8500
Active: 21, Pending: 8499
Active: 22, Pending: 8498
Active: 23, Pending: 8497
Active: 24, Pending: 8496
Active: 25, Pending: 8495
Active: 26, Pending: 8494
Finished SIM_0
Finished SIM_1
Finished SIM_2
Finished SIM_3
Finished SIM_4
Finished SIM_5
Finished SIM_6
Finished SIM_7
Finished SIM_8
Finished SIM_9
Finished SIM_10
Finished SIM_11
Finished SIM_12
Finished SIM_13
Finished SIM_14
Finished SIM_15
Finished SIM_16
Finished SIM_17
Finished SIM_18
Finished SIM_19
Finished SIM_20
Fi

Finished SIM_182
Finished SIM_183
Finished SIM_184
Finished SIM_185
Finished SIM_186
Finished SIM_187
Finished SIM_188
Finished SIM_189
Finished SIM_190
Finished SIM_191
Finished SIM_192
Finished SIM_193
Finished SIM_194
Finished SIM_195
Finished SIM_196
Finished SIM_197
Finished SIM_198
Finished SIM_199
Finished SIM_200
Finished SIM_201
Finished SIM_202
Finished SIM_203
Finished SIM_204
Finished SIM_205
Finished SIM_206
Finished SIM_207
Active: 1, Pending: 8311
Active: 2, Pending: 8310
Active: 3, Pending: 8309
Active: 4, Pending: 8308
Active: 5, Pending: 8307
Active: 6, Pending: 8306
Active: 7, Pending: 8305
Active: 8, Pending: 8304
Active: 9, Pending: 8303
Active: 10, Pending: 8302
Active: 11, Pending: 8301
Active: 12, Pending: 8300
Active: 13, Pending: 8299
Active: 14, Pending: 8298
Active: 15, Pending: 8297
Active: 16, Pending: 8296
Active: 17, Pending: 8295
Active: 18, Pending: 8294
Active: 19, Pending: 8293
Active: 20, Pending: 8292
Active: 21, Pending: 8291
Active: 22, Pending: 

Finished SIM_382
Active: 26, Pending: 8104
Finished SIM_390
Finished SIM_391
Finished SIM_392
Finished SIM_393
Finished SIM_394
Finished SIM_395
Finished SIM_396
Finished SIM_397
Finished SIM_398
Finished SIM_400
Finished SIM_401
Finished SIM_402
Finished SIM_403
Finished SIM_404
Finished SIM_405
Finished SIM_406
Finished SIM_407
Finished SIM_408
Finished SIM_409
Finished SIM_410
Finished SIM_411
Finished SIM_412
Finished SIM_413
Finished SIM_414
Finished SIM_415
Active: 2, Pending: 8103
Active: 3, Pending: 8102
Active: 4, Pending: 8101
Active: 5, Pending: 8100
Active: 6, Pending: 8099
Active: 7, Pending: 8098
Active: 8, Pending: 8097
Active: 9, Pending: 8096
Active: 10, Pending: 8095
Active: 11, Pending: 8094
Active: 12, Pending: 8093
Active: 13, Pending: 8092
Active: 14, Pending: 8091
Active: 15, Pending: 8090
Active: 16, Pending: 8089
Active: 17, Pending: 8088
Finished SIM_399
Active: 17, Pending: 8087
Active: 18, Pending: 8086
Active: 19, Pending: 8085
Active: 20, Pending: 8084
Act

Finished SIM_598
Finished SIM_600
Finished SIM_601
Finished SIM_602
Finished SIM_603
Finished SIM_604
Finished SIM_605
Finished SIM_606
Finished SIM_607
Finished SIM_608
Finished SIM_609
Finished SIM_610
Finished SIM_611
Finished SIM_612
Finished SIM_613
Finished SIM_614
Finished SIM_615
Finished SIM_616
Finished SIM_617
Finished SIM_618
Finished SIM_619
Finished SIM_620
Finished SIM_621
Finished SIM_622
Finished SIM_623
Active: 2, Pending: 7895
Active: 3, Pending: 7894
Active: 4, Pending: 7893
Active: 5, Pending: 7892
Finished SIM_599
Active: 5, Pending: 7891
Active: 6, Pending: 7890
Active: 7, Pending: 7889
Active: 8, Pending: 7888
Active: 9, Pending: 7887
Active: 10, Pending: 7886
Active: 11, Pending: 7885
Active: 12, Pending: 7884
Active: 13, Pending: 7883
Active: 14, Pending: 7882
Active: 15, Pending: 7881
Active: 16, Pending: 7880
Active: 17, Pending: 7879
Active: 18, Pending: 7878
Active: 19, Pending: 7877
Active: 20, Pending: 7876
Active: 21, Pending: 7875
Active: 22, Pending: 

Finished SIM_806
Finished SIM_807
Finished SIM_808
Finished SIM_809
Finished SIM_810
Finished SIM_811
Finished SIM_812
Finished SIM_813
Finished SIM_814
Finished SIM_815
Finished SIM_816
Finished SIM_817
Finished SIM_818
Finished SIM_819
Finished SIM_820
Finished SIM_821
Finished SIM_822
Finished SIM_823
Finished SIM_824
Finished SIM_825
Finished SIM_826
Finished SIM_827
Finished SIM_828
Finished SIM_829
Finished SIM_830
Finished SIM_831
Active: 1, Pending: 7687
Active: 2, Pending: 7686
Active: 3, Pending: 7685
Active: 4, Pending: 7684
Active: 5, Pending: 7683
Active: 6, Pending: 7682
Active: 7, Pending: 7681
Active: 8, Pending: 7680
Active: 9, Pending: 7679
Active: 10, Pending: 7678
Active: 11, Pending: 7677
Active: 12, Pending: 7676
Active: 13, Pending: 7675
Active: 14, Pending: 7674
Active: 15, Pending: 7673
Active: 16, Pending: 7672
Active: 17, Pending: 7671
Active: 18, Pending: 7670
Active: 19, Pending: 7669
Active: 20, Pending: 7668
Active: 21, Pending: 7667
Active: 22, Pending: 

...

Finished SIM_7696
Active: 26, Pending: 797
Finished SIM_7698
Active: 26, Pending: 796
Finished SIM_7700
Active: 26, Pending: 795
Finished SIM_7701
Finished SIM_7703
Finished SIM_7704
Active: 24, Pending: 794
Active: 25, Pending: 793
Active: 26, Pending: 792
Finished SIM_7697
Finished SIM_7699
Finished SIM_7702
Finished SIM_7705
Active: 23, Pending: 791
Active: 24, Pending: 790
Active: 25, Pending: 789
Active: 26, Pending: 788
Finished SIM_7708
Active: 26, Pending: 787
Finished SIM_7706
Finished SIM_7707
Active: 25, Pending: 786
Active: 26, Pending: 785
Finished SIM_7709
Active: 26, Pending: 784
Finished SIM_7710
Active: 26, Pending: 783
Finished SIM_7715
Active: 26, Pending: 782
Finished SIM_7714
Active: 26, Pending: 781
Finished SIM_7711
Finished SIM_7712
Finished SIM_7713
Active: 24, Pending: 780
Active: 25, Pending: 779
Active: 26, Pending: 778
Finished SIM_7716
Active: 26, Pending: 777
Finished SIM_7718
Finished SIM_7720
Active: 25, Pending: 776
Active: 26, Pending: 775
Finished SI

Finished SIM_7891
Active: 26, Pending: 603
Finished SIM_7890
Active: 26, Pending: 602
Finished SIM_7894
Active: 26, Pending: 601
Finished SIM_7892
Active: 26, Pending: 600
Finished SIM_7895
Finished SIM_7900
Active: 25, Pending: 599
Active: 26, Pending: 598
Finished SIM_7893
Active: 26, Pending: 597
Finished SIM_7896
Finished SIM_7897
Finished SIM_7898
Active: 24, Pending: 596
Active: 25, Pending: 595
Active: 26, Pending: 594
Finished SIM_7901
Active: 26, Pending: 593
Finished SIM_7899
Active: 26, Pending: 592
Finished SIM_7902
Finished SIM_7903
Finished SIM_7905
Active: 24, Pending: 591
Active: 25, Pending: 590
Active: 26, Pending: 589
Finished SIM_7906
Finished SIM_7909
Active: 25, Pending: 588
Active: 26, Pending: 587
Finished SIM_7904
Finished SIM_7908
Active: 25, Pending: 586
Active: 26, Pending: 585
Finished SIM_7907
Finished SIM_7911
Active: 25, Pending: 584
Active: 26, Pending: 583
Finished SIM_7910
Active: 26, Pending: 582
Finished SIM_7914
Active: 26, Pending: 581
Finished SI

Finished SIM_8082
Finished SIM_8087
Active: 25, Pending: 410
Active: 26, Pending: 409
Finished SIM_8084
Finished SIM_8085
Finished SIM_8086
Finished SIM_8088
Active: 23, Pending: 408
Active: 24, Pending: 407
Active: 25, Pending: 406
Active: 26, Pending: 405
Finished SIM_8090
Active: 26, Pending: 404
Finished SIM_8089
Finished SIM_8092
Active: 25, Pending: 403
Active: 26, Pending: 402
Finished SIM_8091
Active: 26, Pending: 401
Finished SIM_8093
Active: 26, Pending: 400
Finished SIM_8094
Finished SIM_8096
Finished SIM_8097
Finished SIM_8098
Active: 23, Pending: 399
Active: 24, Pending: 398
Active: 25, Pending: 397
Active: 26, Pending: 396
Finished SIM_8095
Active: 26, Pending: 395
Finished SIM_8099
Active: 26, Pending: 394
Finished SIM_8100
Active: 26, Pending: 393
Finished SIM_8101
Active: 26, Pending: 392
Finished SIM_8102
Finished SIM_8103
Active: 25, Pending: 391
Active: 26, Pending: 390
Finished SIM_8104
Finished SIM_8105
Active: 25, Pending: 389
Active: 26, Pending: 388
Finished SI

Finished SIM_8273
Finished SIM_8275
Finished SIM_8276
Finished SIM_8277
Active: 23, Pending: 219
Active: 24, Pending: 218
Active: 25, Pending: 217
Active: 26, Pending: 216
Finished SIM_8278
Active: 26, Pending: 215
Finished SIM_8279
Finished SIM_8280
Finished SIM_8282
Active: 24, Pending: 214
Active: 25, Pending: 213
Active: 26, Pending: 212
Finished SIM_8281
Active: 26, Pending: 211
Finished SIM_8284
Active: 26, Pending: 210
Finished SIM_8287
Active: 26, Pending: 209
Finished SIM_8285
Finished SIM_8289
Active: 25, Pending: 208
Active: 26, Pending: 207
Finished SIM_8283
Finished SIM_8286
Finished SIM_8288
Finished SIM_8290
Active: 23, Pending: 206
Active: 24, Pending: 205
Active: 25, Pending: 204
Active: 26, Pending: 203
Finished SIM_8291
Active: 26, Pending: 202
Finished SIM_8293
Active: 26, Pending: 201
Finished SIM_8294
Finished SIM_8296
Active: 25, Pending: 200
Active: 26, Pending: 199
Finished SIM_8292
Active: 26, Pending: 198
Finished SIM_8295
Finished SIM_8298
Active: 25, Pendin

Finished SIM_8467
Active: 26, Pending: 25
Finished SIM_8469
Finished SIM_8470
Finished SIM_8473
Active: 24, Pending: 24
Active: 25, Pending: 23
Active: 26, Pending: 22
Finished SIM_8472
Active: 26, Pending: 21
Finished SIM_8468
Finished SIM_8475
Active: 25, Pending: 20
Active: 26, Pending: 19
Finished SIM_8478
Active: 26, Pending: 18
Finished SIM_8474
Active: 26, Pending: 17
Finished SIM_8476
Active: 26, Pending: 16
Finished SIM_8477
Active: 26, Pending: 15
Finished SIM_8481
Active: 26, Pending: 14
Finished SIM_8479
Active: 26, Pending: 13
Finished SIM_8480
Active: 26, Pending: 12
Finished SIM_8482
Active: 26, Pending: 11
Finished SIM_8483
Active: 26, Pending: 10
Finished SIM_8484
Finished SIM_8485
Finished SIM_8486
Finished SIM_8488
Active: 23, Pending: 9
Active: 24, Pending: 8
Active: 25, Pending: 7
Active: 26, Pending: 6
Finished SIM_8489
Finished SIM_8491
Active: 25, Pending: 5
Active: 26, Pending: 4
Finished SIM_8490
Active: 26, Pending: 3
Finished SIM_8493
Active: 26, Pending: 2


Create table

In [2]:
import shutil, os, json, h5py
import pandas as pd
import numpy as np
import  scipy.stats as sps
import pycircstat

file_parts = 'ABCDEFGHIJKLMNOP'
file_part = 0
if swc_path is None:
    morphfiles = os.listdir('../Required_Files/Models/SWCs connected Without/')
else:
    morphfiles = os.listdir(swc_path)
current_vals = pd.read_csv('../Required_files/Calibrated.csv')
current_vals = current_vals.sort_values('Target', ascending=True).reset_index(drop=True).reset_index()
current_vals = current_vals.set_index('cell_id')

def MMR (PhaseList):
    instances = np.array(np.round(PhaseList), dtype = int)
    bin_size = 36  
    gaussian = []
    for i in range(bin_size*2+1):
        gaussian.append(np.exp(-0.5*(i-bin_size)**2/(bin_size/4)**2)/(bin_size*np.sqrt(2*np.pi)/4))
    gaussian = np.array(gaussian)/gaussian[bin_size]
    waveshaped = []
    for i in instances:
        waveshape = np.zeros(360)
        indexes = np.arange(0,len(gaussian))
        indexes = indexes+i-bin_size
        for j in np.arange(0,len(gaussian)):
            if indexes[j]<0:
                indexes[j] +=360
            elif indexes[j]>359:
                indexes[j] -=360
        for j, ind in enumerate(indexes):
            waveshape[ind] = gaussian [j]
        waveshaped.append(waveshape)
    sumation = np.sum(waveshaped, axis = 0)
    sumation = sumation/np.max(sumation)
    return sumation, 1-np.mean(sumation)

all_results = []
all_traces = []
simulation_no = 0
for i in morphfiles:
    if int(i.split('rmed_')[1].split('.sw')[0]) not in range(0,360,Rotation_Steps):
        continue
    if int(i.split('_')[2]) not in current_vals.index and Only_Calibrated:
        continue
    if Manual_picked_model_list is not None:
        if int(i.split('_')[2]) not in Manual_picked_model_list:
            continue
    if supra:
        frequencies = SuP_frequencies
    else:
        if Custom:
            frequencies = SuP_frequencies
        else:
            frequencies = Sub_frequencies
    for f in frequencies:
        if supra:
            if point:
                amplitudes = SuP_amplitudes_P
            else:
                amplitudes = SuP_amplitudes
        else:
            if Custom:
                amplitudes = SuP_amplitudes
            else:
                amplitudes = Sub_amplitudes
        for c in amplitudes:
            if allHOF:
                hof_range = 20
            else:
                hof_range = 1
            for hof in range(hof_range):
                full_path = os.getcwd().replace('\\','/') + '/Simulations/' + \
                                Selected_Sweep + '/SIM_' + str(simulation_no)
                if not Custom:    
                    if supra:
                        try:
                            full_path += '/Data.csv'
                            df = pd.read_csv(full_path)
                        except:
                            if point==0:
                                print("File missing: "+full_path)    
                            simulation_no += 1
                            continue
                        Peak_indices = np.array(df.iloc[:,0])
                        ISIs = Peak_indices[1:]-Peak_indices[:-1]
                        ISIs = 20000 / ISIs
                        mean,std=sps.norm.fit(ISIs)

                        Phases = Peak_indices % (20000/f)
                        Phases = Phases * 360 * f /20000
                        radians = np.asarray(Phases)*np.pi/180
                        mean_phase_rad = pycircstat.descriptive.mean(np.array(radians))
                        mean_phase_angle = mean_phase_rad*(180 / np.pi)
                        mean_pvalue_z=pycircstat.rayleigh(np.array(radians))
                        mean_vector_length = pycircstat.descriptive.vector_strength(np.array(radians))

                        cd, mmr = MMR(Phases)
                        Y, X = np.histogram(Phases, bins=[0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 
                                                          240, 264, 288, 312, 336, 360])
                        Xp =(X[1:] + X[:-1]) / 2
                        Xp = Xp * np.pi / 180
                        normY = np.true_divide(Y, (np.max(Y)))
                        thetas = []
                        cnt = 0.0
                        for cn in range(len(cd)):
                            thetas.append(cnt)
                            cnt = cnt + 2*np.pi/len(cd)
                        all_results.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),
                                            hof, f, c, mean_vector_length, mean_pvalue_z, mean_phase_angle, mmr, 
                                            thetas, cd, Xp, normY, list(Peak_indices), list(ISIs), list(Phases)])
                    else:
                        try:
                            full_path += '/Data.csv'
                            data = pd.read_csv(full_path)
                        except:
                            if point==0:
                                print("File missing: "+full_path)    
                            simulation_no += 1
                            continue
                        row_res = [i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),hof,f,c]
                        for r in data['0']:
                            row_res.append(r)
                        all_results.append(row_res)
                elif axon_parameters is None:
                    full_path += '/Simulation/output/v_report.h5'
                    neuron_data_output_file = full_path
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        data=np.array(single_neuron['data'][()])[2000:5000]
                    fi.close()
                    all_traces.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),
                                       hof,f,c,data])
                else:
                    full_path += '/Simulation/output/v_axon.h5'
                    neuron_data_output_file = full_path
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        if supra:
                            data = np.array(single_neuron['data'][()])[2000:5000]
                        else:
                            data = np.array(single_neuron['data'][()])[19900:20400]
                    fi.close()
                    full_path = full_path.replace('v_axon.h5', 'v_report.h5')
                    neuron_data_output_file = full_path
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        if supra:
                            soma_data=np.array(single_neuron['data'][()])[2000:5000]
                        else:
                            soma_data = np.array(single_neuron['data'][()])[19900:20400]
                    fi.close()
                    all_traces.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),
                                       hof,f,c,soma_data,data])
                simulation_no += 1
                if simulation_no % 2000 == 0:
                    if Custom:
                        if axon_parameters is not None:
                            ff = pd.DataFrame(all_traces)
                            ff.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
                                  'ES_current(uA)', 'Trace(mV)', 'Traces(mV)']
                            ff.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'_'+file_parts[file_part]+'.pkl')
                            del ff
                            del all_results
                            del all_traces
                            all_results = []
                            all_traces = []
                            file_part += 1
if not Custom:
    if supra:
        gf = pd.DataFrame(all_results, columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 
                                                  'ES_frequency(hz)', 'ES_current(uA)', 'mean_vector_length', 
                                                  'mean_pvalue_z', 'mean_phase_angle', 'MMR', 'MMR_X', 'MMR_Y', 
                                                  'VL_X', 'VL_Y', 'Peak_indices', 'ISIs', 'Phases'])
        gf.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'.pkl')
    else:
        df = pd.DataFrame(all_results)
        df.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
                  'ES_current(uA)', 'Raw_Min(mV)', 'Raw_Max(mV)', 'Raw_Mean(mV)',
                  'Raw_Median(mV)', 'Amp_Min(mV)', 'Amp_Max(mV)', 'Amp_Mean(mV)',
                  'Amp_Median(mV)','Amp@40000(mV)', 'Phs_Min(deg)', 'Phs_Max(deg)', 
                  'Phs_Mean(deg)', 'Phs_Median(deg)', 'Phs@30000(deg)', 'Phs@40000(deg)']
        df.to_csv('../Results/Result_Tables/'+ Selected_Sweep +'.csv', index = False)
else:
    ff = pd.DataFrame(all_traces)
    if axon_parameters is None:
        ff.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
              'ES_current(uA)', 'Trace(mV)']
        ff.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'.pkl') 
    else:
        ff.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
              'ES_current(uA)', 'Trace(mV)', 'Traces(mV)']
        ff.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'_'+file_parts[file_part]+'.pkl')    

Erase simulation files

In [ ]:
import shutil, os

shutil.rmtree(os.getcwd().replace('\\','/') + '/Simulations/' + Selected_Sweep)